In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import sklearn 

dataset = pd.read_csv('train.csv')

dataset.describe()

,index,beer/ABV,beer/beerId,beer/brewerId,review/appearance,review/aroma,review/overall,review/palate,review/taste,review/timeUnix,user/ageInSeconds,user/birthdayUnix
count,37500.000000,37500.000000,37500.000000,37500.000000,37500.000000,37500.000000,37500.00000,37500.000000,37500.000000,3.750000e+04,7.856000e+03,7.856000e+03
mean,24951.887573,7.403725,21861.152027,3036.595120,3.900053,3.873240,3.88944,3.854867,3.922440,1.232794e+09,1.176705e+09,2.416303e+08
std,14434.009669,2.318145,18923.130832,5123.084675,0.588778,0.680865,0.70045,0.668068,0.716504,7.190955e+07,3.375514e+08,3.375514e+08
min,0.000000,0.100000,175.000000,1.000000,0.000000,1.000000,0.00000,1.000000,1.000000,9.262944e+08,7.034366e+08,-2.208960e+09
25%,12422.500000,5.400000,5441.000000,395.000000,3.500000,3.500000,3.50000,3.500000,3.500000,1.189194e+09,9.794810e+08,1.433628e+08
50%,24942.500000,6.900000,17538.000000,1199.000000,4.000000,4.000000,4.00000,4.000000,4.000000,1.248150e+09,1.100009e+09,3.183264e+08
75%,37416.750000,9.400000,34146.000000,1315.000000,4.500000,4.500000,4.50000,4.500000,4.500000,1.291330e+09,1.274973e+09,4.388544e+08
max,49999.000000,57.700000,77207.000000,27797.000000,5.000000,5.000000,5.00000,5.000000,5.000000,1.326267e+09,3.627295e+09,7.148988e+08


In [3]:
dataset.drop(['beer/beerId', 'user/ageInSeconds', 'user/birthdayRaw', 'user/birthdayUnix', 'user/gender', 'user/profileName', 'review/timeStruct', 'review/timeUnix'], axis=1, inplace=True)

dataset.dropna( inplace=True)


Standardize ABV

In [4]:
dataset['standardized_abv'] = (dataset['beer/ABV']-dataset['beer/ABV'].mean())/dataset['beer/ABV'].std()

dataset.head()

,index,beer/ABV,beer/brewerId,beer/name,beer/style,review/appearance,review/aroma,review/overall,review/palate,review/taste,review/text,standardized_abv
0,40163,5.0,14338,Chiostro,Herbed / Spiced Beer,4.0,4.0,4.0,4.0,4.0,Pours a clouded gold with a thin white head. N...,-1.036809
1,8135,11.0,395,Bearded Pat's Barleywine,American Barleywine,4.0,3.5,3.5,3.5,3.0,12oz bottle into 8oz snifter.\t\tDeep ruby red...,1.551615
2,10529,4.7,365,Naughty Nellie's Ale,American Pale Ale (APA),3.5,4.0,3.5,3.5,3.5,First enjoyed at the brewpub about 2 years ago...,-1.166230
3,44610,4.4,1,Pilsner Urquell,Czech Pilsener,3.0,3.0,2.5,3.0,3.0,First thing I noticed after pouring from green...,-1.295651
4,37062,4.4,1417,Black Sheep Ale (Special),English Pale Ale,4.0,3.0,3.0,3.5,2.5,A: pours an amber with a one finger head but o...,-1.295651


In [5]:
import string
import re
from nltk.stem import SnowballStemmer, WordNetLemmatizer

from nltk import corpus
from num2words import num2words
from gensim.parsing.preprocessing import STOPWORDS


negators = {'not', 'no', 'never', 'nothing'}
stop = set(STOPWORDS).difference(negators)
stop = stop.union({'will', 'can', 'should', 'shall'})
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()
english_words = set(corpus.words.words())


def text_process(review):
    def is_number(word):
        return word.replace('.','',1).isdigit()
    
    def convert_number(number):
        if is_number(number):
            return num2words(number)
        else:
            return number
    
    def reduce_lengthening(text):
      
        pattern = re.compile(r"(.)\1{2,}")
        return pattern.sub(r"\1\1", text)
    
    
    
    def remove_y(word):
        # remove small words
        if len(word) == 1:
            return ' '
        if word[-1] == 'y' and word[:-1] in english_words:
            return word[:-1]
        elif word[-1] == 'y' and word[-2] == word[-3] and word[:-2] in english_words:
            return word[:-2]
        elif word[-1] == 'y' and word[:-2]+'e' in english_words:
            return word[:-1]
        else:
            return word
    
    def next_word_negation(review):
        is_negated = False
        output = []
        for word in review:
            if word in negators:
                is_negated = True
                continue
            if is_negated and len(word) > 1:
                output.append("neg_" + word)
                is_negated = False
            else:
                output.append(word)
        return output
    
    
    
    review = str(review).lower()
    
    
    
    review = [remove_y(word) for word in review]
    
    # remove stop words
    review = filter(lambda word : word not in stop, review)
    

    review = ' '.join([convert_number(word) for word in review.split()])
   
    review = ' '.join([stemmer.stem(lemmatizer.lemmatize(word, pos='a')) for word in review.split()])
    return review


dataset['processed_beer_style'] = dataset['beer/style'].map(text_process)


dataset['processed_beer_name'] = dataset['beer/name'].map(text_process)


dataset['processed_text'] = dataset['review/text'].map(text_process).astype(str)


dataset.head(5)

,index,beer/ABV,beer/brewerId,beer/name,beer/style,review/appearance,review/aroma,review/overall,review/palate,review/taste,review/text,standardized_abv,processed_beer_style,processed_beer_name,processed_text
0,40163,5.0,14338,Chiostro,Herbed / Spiced Beer,4.0,4.0,4.0,4.0,4.0,Pours a clouded gold with a thin white head. N...,-1.036809,herb spice beer,chiostro,pour cloud gold white head nose floral larg sp...
1,8135,11.0,395,Bearded Pat's Barleywine,American Barleywine,4.0,3.5,3.5,3.5,3.0,12oz bottle into 8oz snifter.\t\tDeep ruby red...,1.551615,american barleywin,beard pat barleywin,twelv oz bottl eight oz snifter deep rub red h...
2,10529,4.7,365,Naughty Nellie's Ale,American Pale Ale (APA),3.5,4.0,3.5,3.5,3.5,First enjoyed at the brewpub about 2 years ago...,-1.166230,american pale ale apa,naught nelli ale,enjoy brewpub year ago final manag bottl sligh...
3,44610,4.4,1,Pilsner Urquell,Czech Pilsener,3.0,3.0,2.5,3.0,3.0,First thing I noticed after pouring from green...,-1.295651,czech pilsen,pilsner urquel,thing notic pour green bottl glass skunk smell...
4,37062,4.4,1417,Black Sheep Ale (Special),English Pale Ale,4.0,3.0,3.0,3.5,2.5,A: pours an amber with a one finger head but o...,-1.295651,english pale ale,black sheep ale special,pour amber finger head onl v strong pour head ...
